In [ ]:
using Revise


using LinearAlgebra, QuadGK, Roots, FFTW, FastGaussQuadrature
using VlasovSolvers
import VlasovSolvers: advection!
import VlasovSolvers: samples, Particles, PIC_step!, ParticleMover, kernel_poisson!, kernel_gyrokinetic!
using ProgressMeter
using Plots
# pgfplotsx()


using LaTeXStrings

# Quadrature rules

In [ ]:
struct RectangleRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function RectangleRule(len, start, stop)
        points = LinRange(start, stop, len+1)[1:end-1]
        s = step(points) 
        weights = [s for _ = 1:len]
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct TrapezoidalRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function TrapezoidalRule(len, start, stop)
        points = LinRange(start, stop, len)[1:end]
        s = step(points) 
        weights = [s for _ = 1:len]
        weights[1] /= 2
        weights[end] /= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct SimpsonRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function SimpsonRule(len, start, stop)
        # make sure the number of points is uneven
        if len % 2 == 0
            len += 1
        end
        points = LinRange(start, stop, len)
        s = step(points) 
        weights = s/3 .* ones(len)
        weights[2:2:end-1] .*= 4
        weights[3:2:end-2] .*= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct GaussHermiteRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussHermiteRule(len, start, stop)
        points, weights = gausshermite(len)
        weights .*= exp.(points.^2)
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussLegendreRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLegendreRule(len, start, stop)
        points, weights = gausslegendre(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussRadauRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussRadauRule(len, start, stop)
        points, weights = gaussradau(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussLobattoRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLobattoRule(len, start, stop)
        points, weights = gausslobatto(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct KronrodRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function KronrodRule(len, start, stop)
        pts, w, _ = kronrod(len)
        weights = []
        points = []
        for i = 1:len
            push!(points, pts[i])
            push!(weights, w[i])
            push!(points, -pts[i])
            push!(weights, w[i])
        end
        push!(points, pts[end])
        push!(weights, w[end])
        
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        len = 2*len + 1
        new(len, start, stop, points, weights)
end
    end

In [ ]:
# Only works for rectangle and trapezoidal rules
function projection_onto_grid!(grid_dst, meshx, meshv, X, V_, W)
    meshxstep = meshx[2] - meshx[1]
    meshvstep = meshv[2] - meshv[1]
    grid_dst .= 0


    # Periodic Boundary conditions on velocity
    V = copy(V_)
    V[findall(v -> v >= meshv[end],  V)] .-= meshv[end] - meshv[1]
    V[findall(v -> v < meshv[1],  V)] .+= meshv[end] - meshv[1]
    
    for ipart = 1:length(X)
        idxgridx = Int64(fld(X[ipart],            meshxstep)) + 1
        idxgridv = Int64(fld(V[ipart] - meshv[1], meshvstep)) + 1
        idxgridxp1 = idxgridx<length(meshx) ? idxgridx+1 : 1
        idxgridvp1 = idxgridv<length(meshv) ? idxgridv+1 : 1
        
        tx = (X[ipart]             - (idxgridx-1) * meshxstep) / meshxstep
        tv = (V[ipart] - meshv[1]  - (idxgridv-1) * meshvstep) / meshvstep

        # println((idxgridv, V[ipart]))
        grid_dst[idxgridx  , idxgridv  ] += W[ipart] * (1-tx) * (1-tv)
        grid_dst[idxgridx  , idxgridvp1] += W[ipart] * (1-tx) * tv
        grid_dst[idxgridxp1, idxgridvp1] += W[ipart] * tx     * tv
        grid_dst[idxgridxp1, idxgridv  ] += W[ipart] * tx     * (1 - tv)
    end
end

# SL classique

In [ ]:
"""
    hmf_poisson!(fᵗ    :: Array{Complex{Float64},2},
                 mesh1 :: OneDGrid,
                 mesh2 :: OneDGrid,
                 ex    :: Array{Float64})

    Compute the electric hamiltonian mean field from the
    transposed distribution function

"""
function hmf_poisson!(fᵗ::Array{Complex{Float64},2},
        mesh1::OneDGrid,
        mesh2::OneDGrid,
        ex::Array{Float64}; K=1)

    n1 = mesh1.len
    rho = mesh2.step .* vec(sum(fᵗ, dims=1)) # ≈ ∫ f(t,x_i,v)dv, i=1, ..., n1
    kernel = zeros(Float64, n1)
    ker = -(mesh1.stop - mesh1.start) / (2π)
    for k=1:K
        kernel[1+k]   =  ker / k    # fourier mode  1
        kernel[end - (k-1)] = -ker / k    # fourier mode -1
    end
    ex .= real(ifft(fft(rho) .* 1im .* kernel))
end

function solve_SL!(nsteps, dt, f, mesh1, mesh2, kx; plotting=false::Bool)
    n1, n2 = size(f)
    fᵗ = zeros(Complex{Float64}, (n2,n1))
    transpose!(fᵗ, f)

    energy² = Array{Float64}(undef, nsteps)
    etot² = Array{Float64}(undef, nsteps)
    momentum = Array{Float64}(undef, nsteps)

    ex = zeros(Float64, n1)
    hmf_poisson!(fᵗ, mesh1, mesh2, ex)
    advection!(fᵗ, mesh2, ex, 0.5dt)

    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    
    animation = @animate for istep = 1:nsteps
        energy²[istep] = sum(ex.^2) * mesh1.step
        etot²[istep] = (energy²[istep] + sum(mesh2.points'.^2 .* real(f)) * mesh1.step * mesh2.step) / 2
        momentum[istep] = sum(sum(real(f), dims=1) .* mesh2.points) * mesh1.step * mesh2.step
    
        advection!(f, mesh1, mesh2.points, dt)
        transpose!(fᵗ, f)
        hmf_poisson!(fᵗ, mesh1, mesh2, ex)
        advection!(fᵗ, mesh2, ex, dt)
        transpose!(f, fᵗ) 
        
        if plotting
            plot(mesh1.points, mesh2.points, real(f)', size=(500, 500), st=:surface, camera=(0, 90))
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        
        ProgressMeter.next!(progression)
    end when plotting
    if plotting
        return sqrt.(energy²), sqrt.(etot²), momentum, ex, animation
    else
        return sqrt.(energy²), sqrt.(etot²), momentum, ex, nothing
    end
end

# PIC solver

In [ ]:
function solve_PIC!(nsteps, dt, particles, meshx, kx; plotting=false::Bool, kernel=kernel_poisson!)
    energy_elec² = Array{Float64}(undef, nsteps)
    energy_tot²= Array{Float64}(undef, nsteps)
    momentum = Array{Float64}(undef, nsteps)

    L = meshx.stop
    np = particles.nbpart

    pmover = ParticleMover(particles, meshx, kx, 1, dt)

    if plotting
        widthx = -(-)(extrema(quadrulex.points)...)
        widthv = -(-)(extrema(quadrulev.points)...)
        scale = 25
    end
    
    progression = ProgressMeter.Progress(nsteps, desc="Loop in time: ", showspeed=true)
    animation = @animate for istep = 1:nsteps
        if plotting
            scale = 25
            plot(vec(p.x), vec(p.v), vec(p.wei), seriestype=:scatter, markersize=sqrt(widthx * widthv * scale^2  / (nx*nv) / π), camera=(0, 90), markerstrokecolor="white", markerstrokewidth=0, label="", zcolor=vec(p.wei), c=:rainbow,aspect_ratio=:equal, size=(widthx, widthv).*scale)
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end

        eelec², mom, etot² = PIC_step!(p, pmover; kernel=kernel)
        energy_elec²[istep] = eelec²
        momentum[istep] = mom
        energy_tot²[istep] = etot² 
        
        ProgressMeter.next!(progression)
    end when plotting
    if !plotting
        animation = nothing
    end
    return sqrt.(energy_elec²), momentum, sqrt.(energy_tot²), animation
end

# INPUTS

## Landau damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.001
μ = 0.5
β = 1.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * (v-μ)^2 / 2) / √(2π/β) #+ exp(-((x-L/2)^2 + (v-5)^2) / (2 * 0.1)) + (1 <= x <= 2) * (-7 <= v <= -5)
castest = "Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

## Two-Stream Instability

### Première version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = -0.001
β₁ = 1
β₂ = 1
v0 = 2.4
μ₁ =  v0
μ₂ = -v0
f(x,v) = (1 + ϵ * cos(kx * x)) * (exp(-β₁*(v-μ₁)^2 / 2) + exp(-β₁*(v-μ₂)^2 / 2)) / (√(2π)) * 0.5
castest = "TSI\n(kx, β₁, β₂, μ₁, μ₂) = $((kx, β₁, β₂, μ₁, μ₂))"

### Seconde version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = -0.05
β = 1
μ =  0
f(x,v) = (1 + ϵ * cos(kx * x)) * v^2 * exp(-β*(v-μ)^2 / 2) / (√(2π))
castest = "TSI\n(kx, β, μ) = $((kx, β, μ))"

## Bump-on-Tail

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = 0.05
β₁ = 1
β₂ = 4
μ₁ = 0
μ₂ = 4.5
nb = 0.2
np = 0.9
f(x,v) = (1 + ϵ * cos(kx*x)) * (np*exp(-β₁*(v-μ₁)^2 /2) + nb*exp(-β₂*(v-μ₂)^2 / 2)) / √(2π)
castest = "Bump on tail\n(kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np) = $((kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np))"

## Strong Landau Damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.5
β = 1.0
μ = 0.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * v^2 / 2) / √(2π/β)
castest = "Strong Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

# Inputs indépendants du cas test

In [ ]:
dev = CPU()

nstep = 1000
dt = 0.1

nx = 128
nv = 129

vmax = 9.

meshx = OneDGrid(dev, nx, 0, L);
meshv = OneDGrid(dev, nv, -vmax, vmax);

quadX = RectangleRule
quadV = RectangleRule

quadrulex = quadX(nx, 0, L);
quadrulev = quadV(nv, -vmax, vmax);
# quadrulev = quadV(nv, μ - 5/√β, μ + 5/√β)


T = NaN;

In [ ]:
gridv = -vmax:0.01:vmax

ex_quadrulev = quadV(nv, -vmax, vmax);
ex_quadrulev2 = quadV(nv, μ - 5/√β, μ + 5/√β)
ex_quadrulev3 = quadV(nv, μ - 4/√β, μ + 4/√β)

plot(gridv, f.(π/2, gridv))
plot!(ex_quadrulev.points, zeros(ex_quadrulev.len) .+ 0.01, seriestype=:scatter, markerstrokewidth=0, label="grille totale")
plot!(ex_quadrulev2.points, zeros(ex_quadrulev2.len) .+ 0.025, seriestype=:scatter, markerstrokewidth=0, label="±5σ")
plot!(ex_quadrulev3.points, zeros(ex_quadrulev3.len) .+ 0.040, seriestype=:scatter, markerstrokewidth=0, label="±4σ")

In [ ]:
gr() 
plot(quadrulex.points, quadrulev.points', f.(quadrulex.points, quadrulev.points')', st=:surface, camera=(80, 30))
xlabel!("x")
ylabel!("v")
# savefig("gif/gyrokinetic_example.png")

# Simulations

In [ ]:
gsl  = zeros(Complex{Float64}, (nx,nv));
@. gsl = f.(meshx.points, meshv.points');
@time E_elecsl, E_totsl, momentumsl, _, animationsl = solve_SL!(nstep, dt, gsl, meshx, meshv, kx; plotting=false);

In [ ]:
# gif(animationsl, "animationsl.gif")

# PIC interpretation

In [ ]:
npgsl = nx*nv
x0 = vec(repeat(quadrulex.points, 1, quadrulev.len)) 
v0 = vec(repeat(quadrulev.points', quadrulex.len, 1))
wei = vec(f.(quadrulex.points, quadrulev.points') .* quadrulex.weights .* quadrulev.weights')
p = Particles(x0, v0, wei, npgsl);
@time E_eleccarac, momentumcarac, E_totcarac, animationcarac = solve_PIC!(nstep, dt, p, meshx, kx, plotting=false; kernel=kernel_poisson!);

In [ ]:
if animationcarac != nothing
    gif(animationcarac, "gif/gyrokinetic_example.mp4");
end

# Classical PIC

In [ ]:
np = Int64(2e5)
(x0, y0, wei) = samples(np, kx, ϵ, μ, β)
p = Particles(x0, y0, wei, np);
@time E_elecpic, momentumpic, E_totpic, animation = solve_PIC!(nstep, dt, p, meshx, kx;plotting=false);

# Plots

In [ ]:
t = (1:nstep) .* dt

# energies
# plot(t .+ dt, E_elec, label=L"E_{elec, carac},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true, size=(400, 200).*2)
# plot!(t, E_elecsl, label=L"E_{elec, SL},\quad dt="*"$(dt)", ls=:dash)
# plot!(t .+ dt, E_elec2, label=L"E_{elec, carac, refined},\quad dt="*"$(dt)")
# plot!(t, energy_hamil_elec_gsl, label=L"E_{elec, PIC charac},\quad dt="*"$(dt)")
# plot!(t, energy_hamil_elec, label=L"E_{elec, PIC},\quad dt="*"$(dt)")
# plot!(t, E_tot, label=L"E_{tot}", scaley=:log10)

# log(Energies)
plot(legend=:top, minorgrid=true, size=(800, 400))
plot!(t .+ dt, log.(E_eleccarac), label=L"\log(E_{elec, carac}),\quad dt="*"$(dt)")
plot!(t, log.(E_elecsl), label=L"\log(E_{elec, SL}),\quad dt="*"$(dt)", ls=:dash, lw=1)
# plot!(t, log.(E_tot),       label=L"\log(E_{tot}),\quad dt="*"$(dt)")
# plot!(t, log.(E_elecpic), label=L"$\log(E_{elec, PIC}),\quad$ dt="*"$(dt), "* L"$n_p$ ="*"$(np)", ls=:dot)
# plot!(t, log.(energy_from_projection), label=L"\log(E_{pic, PIC}),\quad dt="*"$(dt)")
# plot!(t, log.(energy_elec_from_phi), label=L"\log(E_{from\,\Phi, PIC}),\quad dt="*"$(dt)")


# ============== #

# Landau damping (kx=0.5):
# plot!(x->-0.1533x - 5.6, label="Damping attendu (-0.1533)")
# E_th = abs.(4ϵ * 0.3677 .* exp.(−0.1533 .* t) .* cos.(1.4156.*t .−0.5326245)) * sqrt(L/2)
# plot!(t, log.(E_th),label="Energie theorique")
# Landau damping (kx=0.4):
# plot!(x->-0.0661x - 5.3, label="Damping attendu (-0.0661)")
# E_th = 0.002.*0.42466.*abs.(cos.(1.285.*t .-0.33577)).*exp.(-0.0661.*t) # expression du bouquin, pas correcte
# E_th = abs.(4*ϵ*0.424666*exp.(-0.0661 .* t) .* cos.(1.2850 .* t .- 0.3357725) * sqrt(L/2)) # issue des calculs du bouquin en calculant correctement √(∫sin(0.5x)^2dx)
# plot!(t, log.(E_th),label="Energie theorique", ls=:dashdotdot)

# TSI (k,v0) = (0.2, 1.3):
# plot!(t, -0.001t .- 4.2, label=L"y=0.001t - 5.0")
# TSI (k,v0) = (0.2, 2.4):
# plot!(t, 0.2258t .- 6.4, label=L"y=0.2258t - 8.4")
# TSI (k,v0) = (0.2, 3):
# plot!(t, 0.2845t .- 6.1, label=L"y=0.2845t - 8.2")

# Strong Landau damping
# plot!(t, -0.285473t .+ 1, label=L"y=-0.285473t + 1")
# plot!(t, 0.086671t .- 3.7, label=L"y=0.086671t - 3.7")

title!(castest * "\n($(quadX)($(nx)), $(quadV)($(nv)))")
# xlabel!("t (T=$(T))")
xlabel!("time")

In [ ]:
fn = "methode_carac/test_pgfplots"
savefig("/Users/ylehenaf/Documents/latex/imgs/tex/$(fn).tikz")

# Plot annotations

In [ ]:
p = 2π / 1.4156
vline!(vcat([4 + i*p for i=0:4], [49.5 + i*p for i=0:3]), label="")
# vline!([28.1 + i*p for i=-4:0], label="")

In [ ]:
plot!([49.5, 49.5+p], [-10, -10], arrow=arrow(:closed, :both), label="")
annotate!((51.5, -10.3, text(L"\frac{2\pi}{1.4156}", :top, 10)))

# Quantités conservatives (SL Généralisé)

## Variation d'énergie totale

In [ ]:
p11 = plot(t, E_totcarac, label="", minorgrid=true)
title!(p11, "Énergie totale (carac)")
p12 = plot(t, (E_totcarac .- E_totcarac[1]) ./ E_totcarac[1], label="", minorgrid=true)
title!(p12, "Relative (carac)")
p21 = plot(t, E_totsl, label="", minorgrid=true)
title!(p21, "Énergie totale (SL)")
p22 = plot(t, (E_totsl .- E_totsl[1]) ./ E_totsl[1], label="", minorgrid=true)
title!(p22, "Relative (SL)")
p=plot(p11, p12, p21, p22, size=(800, 600), layout=(2, 2))

In [ ]:
p11 = plot(t, E_totpic, label="")
title!(p11, "Énergie totale (PIC)")
p12 = plot(t, (E_totpic .- E_totpic[1]) ./ E_totpic[1], label="")
title!(p12, "Relative (PIC)")
p=plot(p11, p12, size=(800, 600), layout=(2, 1))

## Variations du moment

In [ ]:
p11 = plot(t, momentumcarac, label="", minorgrid=true)
title!(p11, "Momentum (carac)")
p12 = plot(t, (momentumcarac .- momentumcarac[1]) ./ momentumcarac[1], label="", minorgrid=true)
title!(p12, "Relatif (carac)")
p21 = plot(t, momentumsl, label="", minorgrid=true)
title!(p21, "Momentum (SL)")
p22 = plot(t, (momentumsl .- momentumsl[1]) ./ momentumsl[1], label="", minorgrid=true)
title!(p22, "Relatif (SL)")
p=plot(p11, p12, p21, p22, size=(800, 600), layout=(2, 2))

In [ ]:
p1 = plot(t, momentumpic, label="", minorgrid=true)
title!(p1, "Momentum (PIC)")
p2 = plot(t, (momentumpic .- momentumpic[1]) ./ momentumpic[1], label="", minorgrid=true)
title!(p2, "Relative (PIC)")
p=plot(p1, p2, size=(800, 600), layout=(2, 1))

# Comparison with VlasovSolver's Bump on tail

In [ ]:
using VlasovSolvers
using Plots
using LaTeXStrings

In [ ]:
dev = CPU()                  # device
stepper = StrangSplitting()  # timestepper
dt = 0.05                     # timestep
nsteps = 8000                # total number of time-steps

α   = 0.04
kx  = 0.3

n1, n2 = 32, 64
x1min, x1max = 0.0, 20π
x2min, x2max = -9., 9.

mesh1 = OneDGrid(dev, n1, x1min, x1max)
mesh2 = OneDGrid(dev, n2, x2min, x2max)

fbot = DistributionFunction( mesh1, mesh2 )


for (i,x) in enumerate(mesh1.points), (j,v) in enumerate(mesh2.points)
    fbot.values[i,j]  = (1.0+α*cos(kx*x)) / (10*sqrt(2π)) * (9*exp(-0.5*v^2)+2*exp(-2*(v-4.5)^2))
end


In [ ]:
prob = VlasovProblem(fbot, BSLSpline(5), dev)

sol = VlasovSolvers.solve!(prob, stepper, dt, nsteps );

In [ ]:
plot(dt .* (1:nsteps), exp.(sol).^2, size=(600, 300))

# Comparison of quadratures on Bump on Tail

In [ ]:
dev = CPU()

nstep = 2000
dt = 0.05

nx = 32
nv = 64

vmax = 9.

meshx = OneDGrid(dev, nx, 0, L);
meshv = OneDGrid(dev, nv, -vmax, vmax);
gsl  = zeros(Complex{Float64}, (nx,nv));


rules = [RectangleRule, TrapezoidalRule, GaussHermiteRule, GaussLegendreRule, GaussRadauRule, GaussLobattoRule, KronrodRule]
t = (1:nstep) .* dt


for quadX = rules, quadV = rules
    quadrulex = quadX(nx, 0, L);
    quadrulev = quadV(nv, -vmax, vmax);

    @. gsl = f.(meshx.points, meshv.points');
    E_elecsl, E_totsl, _, animationsl = solve_SL!(nstep, dt, gsl, meshx, meshv, kx; plotting=false);
    E_elec, E_tot, animation, X, V, F = generalized_SL(nstep, dt, quadrulex, quadrulev, kx, f, plotting=plotornot);

    plot(t, E_elec, label=L"E_{elec},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true, size=(400, 200).*2)
    plot!(t, E_elecsl, label=L"E_{elec, SL},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true)

    title!(castest * "\n($(quadX)($(nx)), $(quadV)($(nv)))")
    xlabel!("t")
    savefig("/Users/ylehenaf/Documents/latex/imgs/methode_carac/comparison_quadratures/$(quadX)_$(quadV).png")
end
println("Fini")

# Divers

In [ ]:
ptsX = TrapezoidalRule(32, 0, L);
ptsV = GaussHermiteRule(32, -vmax, vmax);
scatter(repeat(ptsX.points, 1, 32), repeat(ptsV.points', 32, 1), label="")
# for x = ptsX.points, v = ptsV.points
#     sc((x, v))
# end
title!("pts quadrature")

interpolation 2D (NUFFT)

euler 2D

approximation sparse d'une somme (hyperbolic cross)

HMF 4D (2Dx * 2Dv)

Conservation énergie globale après discrétisation du schéma

## Projection onto grid / Interpolation

In [ ]:
E_elec, E_tot, momentum, l2norm, animation, X, V, F = generalized_SL(50, dt, quadrulex, quadrulev, kx, f; plotting=false);

In [ ]:
spline_proj = Dierckx.Spline2D(X, V, vec(f.(quadrulex.points, quadrulev.points')); s=1e-5);
grid_proj = evalgrid(spline_proj, quadrulex.points, quadrulev.points);

In [ ]:
plot(quadrulex.points, quadrulev.points, grid_proj', st=:surface, camera=(0, 90))

## Visualisation des points de quadrature

In [ ]:
fv(v) = f(π/2, v)
plot(fv, -vmax, vmax)
quadrulev1 = quadV(Int(fld(nv, 2)), μ₁ - 5/√β₁, μ₁ + 5/√β₁)
scatter!(quadrulev1.points, zeros(quadrulev1.len) .+ 0.05, strokecolor=0)
quadrulev2 = quadV(Int(fld(nv, 2)), μ₂ - 5/√β₂, μ₂ + 5/√β₂)
scatter!(quadrulev2.points, zeros(quadrulev2.len) .+ 0.1, strokecolor=0)

quadrulevref = quadV(nv, -vmax, vmax)
scatter!(quadrulevref.points, zeros(quadrulevref.len), strokecolor=0)

In [ ]:
abs(sum(fv.(quadrulev.points) .* quadrulev.weights)-1), abs(sum(fv.(newquadrulev.points) .* newquadrulev.weights)-1)

In [ ]:
E_eleccarac